# Iron Supported, Nitrogen-doped, Graphene (And Related Systems) <a name="head"></a>
***

#### Table of Contents

-   [Import Python Modules](#import-modules)
-   [Free Energy Diagram](#fed_all)
-   [Free Energy Corrections](#fe_corrections)

## Import Modules <a name="import-modules"></a>

In [18]:
# iPython Settings
%load_ext autoreload
%autoreload 2

import sys
import copy

import numpy as np
import pandas as pd

from ase.visualize import view

import plotly.plotly as py
import plotly.graph_objs as go

# My Modules ******************************************************************
from dft_job_automat.job_analysis import DFT_Jobs_Analysis
from dft_job_automat.job_types_classes.dft_methods import DFT_Methods
from orr_reaction.orr_methods import ORR_Free_E_Plot, calc_ads_e
from orr_reaction.orr_methods import lowest_e_path, df_calc_adsorption_e, plot_all_states
from energetics.dft_energy import Element_Refs

pd.options.mode.chained_assignment = None

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [20]:
bias = 0.

# Bare Fe
bare_slab_False = -30772.862053560006
bare_slab_True = -30774.394355700322
bare_slab_dict_Fe = {True: bare_slab_True, False: bare_slab_False}

# Bare Graphene
bare_slab_False = -947.9120007283367
bare_slab_True = -947.9121850854704
bare_slab_dict_Graphene = {True: bare_slab_True, False: bare_slab_False}

# Fe-supported Graphene
bare_slab_True = -31722.43520417364
bare_slab_False = -31721.412152900266
bare_slab_dict_Fe_Graph = {True: bare_slab_True, False: bare_slab_False}

# Fe-supported N-Graphene
# /scratch/users/flores12/03_graph_N_Fe/01_opt_struct/N_doped_graph_Fe/no_spin/01_N_trifold/_4
# N in trifold site
bare_slab_NGraph_Fe = -31839.06523325643

# C in trifold site
tmp = -31838.6148526

# Refs = Element_Refs()
# oxy_ref = Refs.E_O_ref.gibbs_e
# hyd_ref = Refs.E_H_ref.gibbs_e

oxy_ref = -443.5879856
hyd_ref = -16.47806172

color_list = [
    "rgb(113,166,190)",
    "rgb(145,209,79)",
    "rgb(124,78,196)",
    "rgb(203,169,87)",
    "rgb(200,72,150)",
    "rgb(130,204,159)",
    "rgb(190,82,63)",
    "rgb(80,51,82)",
    "rgb(81,92,54)",
    "rgb(192,151,188)",
    ]

fe_corr_dict = {
    "h2o": 0.528688009,
    "o": 0.068715186,
    "oh": 0.265727434,
    "ooh": 0.351189818,
    }

dir_list = ["Fe_slab", "graphene", "graph_Fe", "N_graph_Fe"]

sys_dict = [
    {"folder": "Fe_slab", "bare_ref": bare_slab_dict_Fe},
    {"folder": "graphene", "bare_ref": bare_slab_dict_Graphene},
    {"folder": "graph_Fe", "bare_ref": bare_slab_dict_Fe_Graph},
    {"folder": "N_graph_Fe", "bare_ref": bare_slab_NGraph_Fe},
    ]

df_sys = pd.DataFrame(sys_dict)

Jobs_list = []
for index, row in df_sys.iterrows():
    folder_name = row["folder"]

    Jobs = DFT_Jobs_Analysis(
        update_job_state=False,
        job_type_class=None,
        load_dataframe=True,
        working_dir=folder_name + "/1STEP",
        )

    df = Jobs.filter_early_revisions(Jobs.data_frame)

    if folder_name == "N_graph_Fe":
        df_calc_adsorption_e(df, oxy_ref, hyd_ref, row["bare_ref"], bare_slab_var="spinpol")
    else:
        df_calc_adsorption_e(df, oxy_ref, hyd_ref, row["bare_ref"], bare_slab_var="spinpol", corrections_mode="corr_dict", corrections_dict=fe_corr_dict)
        
    df["system"] = folder_name
    Jobs.df = df   
    Jobs_list.append(Jobs)

# Free Energy Corrections <a name="fed_all"></a>
***

In [63]:
master_data_list = []
for Jobs_i in Jobs_list:
    df = Jobs_i.df

    if "gibbs_correction" not in list(df):
        continue
    
    groupby = ["adsorbate"]
    data_master = {}
    
    data_list_j = []
    for group_i in df.groupby(groupby):
        group_i = group_i[1][group_i[1]["gibbs_correction"] > 0.]

        gibbs_corr_i = group_i["gibbs_correction"].tolist()

        data = [
            go.Bar(
                x=group_i['adsorbate'], # assign x as the dataframe column 'x'
                y=group_i['gibbs_correction'],
                width = 0.25,)
        ]

        data_list_j = data_list_j + data
        
        # Plot this along with the bar graph
        mean_fe_corr = group_i["gibbs_correction"].mean()

    master_data_list += data_list_j
    
py.iplot(master_data_list, filename='pandas-bar-chart')